<a href="https://colab.research.google.com/github/cheng1103/Test/blob/main/Cnyes_category_fast.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1><span style="font-size:20px"><strong>鉅亨網新聞主題爬蟲（快速版，用於Colab平台）</strong></span></h1>

<p>可自訂主題新聞與日期區間爬取新聞內文，並可設定是否爬取頭條以外的新聞<br />
<span style="color:null"><strong>輸出資料的欄位依序為：</strong></span> <strong>張貼時間、新聞編號(網址)、版別、標題、頭條標記、內文、股號(如果有)</strong> <strong>。資料格式為csv檔，並以UTF8編碼，直接存入Google雲端硬碟</strong></p>

<p>支援的分類新聞目錄：</p>

<p>1.台股 V<br />
&nbsp; a.台灣政經 V<br />
&nbsp; b.台股盤勢 V<br />
&nbsp; c.台股盤勢 V<br />
&nbsp; d.台股盤勢 V</p>

<p>2.國際股 V<br />
&nbsp; a.美股 V<br />
&nbsp; b.美股雷達 V<br />
&nbsp; c.歐亞股 V<br />
&nbsp; d.國際政經 V</p>

<p>3.陸港股 V</p>

<p>4.外匯 V</p>

<p>5.期貨 V</p>

<p>6.房產 V</p>

<p>7.基金 V</p>

<p>8.區塊鏈 V</p>


# 新聞類別與查詢區間設定


In [23]:
#.設定區域
#==============================================
#頭條設定 
# 1表示限定頭條； 0表示不限 
Headline = 1

#輸入類別
c = '台股'

#查詢區間
s_time = "2021-07-01 00:00:00" #開始日期 
e_time = "2021-12-31 00:00:00" #結束日期

#檔名設定
file_name = 'original_data'

#==============================================

data = []

kind = {
  '台股':'tw_stock',
  '台灣政經':'tw_macro',
  '台股盤勢':'tw_quo',
  '專家觀點':'stock_report',
  '台股新聞':'tw_stock_news',

  '國際股':'wd_stock',
  '美股':'us_stock',
  '美股雷達':'us_stock_live',
  '歐亞股':'eu_asia_stock',
  '國際政經':'wd_macro',

  '陸港股':'cn_stock',
  '外匯':'forex',
  '期貨':'future',
  '房產':'cnyeshouse',
  '基金':'fund',
  '區塊鏈':'bc'

}

c = kind.get(c)


#頭條篩選
if Headline == 1 :
  CH = '&isCategoryHeadline=1'
else :
  CH = ''

# 安裝並載入套件

In [15]:
!pip install fake_useragent
from fake_useragent import UserAgent
import requests
from bs4 import BeautifulSoup
from lxml import etree

import random
import time
import datetime
import json
import pandas as pd
import re
#%reset -f

# 自定義程式

In [18]:
def url_list(s_time,e_time):
  
  #==============================================
  #時間區間設定
  s_time = int(time.mktime(time.strptime(s_time, "%Y-%m-%d %H:%M:%S")))-28800 # 轉成時間戳
  e_time = int(time.mktime(time.strptime(e_time, "%Y-%m-%d %H:%M:%S")))-28800

  #==============================================
  p = 1
  l_page = 9
  data = pd.DataFrame()

  while p <= l_page :

    headers = UserAgent().random #隨機請求表頭
    
    url = 'https://api.cnyes.com/media/api/v1/newslist/category/'+c+'?startAt='+str(s_time)+'&endAt='+str(e_time)+CH+"&limit=30&page="+str(p)
    req = requests.get(url, headers={"User-Agent":headers})
    req = req.json()

    #最末頁之路徑
    if p == 1:
      l_page = req["items"]["last_page"] 

    req_data = req["items"]["data"]
    temp = pd.DataFrame.from_dict(req_data)
    
    #擷取資料
    publishAt = temp['publishAt']
    newsId_url = temp['newsId']
    category = temp['categoryName']
    title = temp['title']
    Headline = temp['isCategoryHeadline']
    stock = temp['stock']
    content = temp['content']

    temp_data = pd.concat([publishAt, Headline, category, title, content, stock, newsId_url], axis=1)
    temp_data = reFormat(temp_data)

    data = pd.concat([data, temp_data], ignore_index=True)
    


    #休眠
    t = random.uniform(0.005, 0.1) #隨機數
    time.sleep(t) #暫停

    p = p+1

    if p%20 == 0:
      print(p/l_page*100)
      data.drop_duplicates(subset='newsId', keep='first', inplace=True) #移除重複資料
      t = random.uniform(0.2, 1) #隨機數
      time.sleep(t)


  data.drop_duplicates(subset='newsId', keep='first', inplace=True) #移除重複資料
  return(data)



In [17]:
def reFormat (temp_data):
  for r in range(len(temp_data)):

    #擷取文字
    c = temp_data['content'][r]
    c = etree.HTML(c).xpath('string(.)')
    c = etree.HTML(c).xpath('//*[not(figcaption)]/p/text()') #####
    c = "".join(c)
    c = c.replace('\n', '').replace('\xa0', '')

    temp_data['content'][r] = c


    #擷取股票代號
    temp_data['stock'][r] = ','.join(temp_data['stock'][r])
    if len(temp_data['stock'][r]) == 0 :
      temp_data['stock'][r] = '0'

    #轉換時間格式
    struct_time = time.localtime(temp_data['publishAt'][r]) # 轉成時間元組
    temp_data['publishAt'][r] = time.strftime("%Y/%m/%d %H:%M", struct_time) # 轉成字串

  return(temp_data)

In [ ]:
  headers = UserAgent().random
  url = 'https://api.cnyes.com/media/api/v1/newslist/category/tw_stock?startAt=1640966400&endAt=1641139199&limit=30'
    
  req = requests.get(url, headers={"User-Agent":headers})
  req = req.json()
  req_data = req["items"]["data"]
  temp = pd.DataFrame.from_dict(req_data)

  publishAt = temp['publishAt']
  newsId_url = temp['newsId']
  category = temp['categoryName']
  title = temp['title']
  Headline = temp['isCategoryHeadline']
  stock = temp['stock']
  content = temp['content']

  temp_data = pd.concat([publishAt, Headline, category, title, content, stock, newsId_url], axis=1)
  
  c = temp_data['content'][11]

'''
  c = etree.HTML(text=c)
  c = c.xpath('string(.)')
  pattern = re.compile(r'<[^>]+>',re.S)
  c = pattern.sub('', c)
  c = c.replace('\n', '').replace('&nbsp', '')

''' 
  #c = temp_data['content'][r]
  c = etree.HTML(c).xpath('string(.)')
  c = etree.HTML(c).xpath('//*[not(figcaption)]/p/text()')
  c = "".join(c)
  c = c.replace('\n', '').replace('\xa0', '')



  


  # 10 11


  
  
  c

'「We are family」、「We are Taiwan」字樣出現在台北 101 跨年到數動畫影片，期盼凝聚臺灣人民的心，迎向 2022。／中國信託銀行提供'

# 執行程式

In [24]:
data = url_list(s_time,e_time)
data = data.sort_values(by='publishAt', ascending=True)
data = data.reset_index(drop=True) 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/li

6.191950464396285
12.38390092879257
18.575851393188856
24.76780185758514
30.959752321981426
37.15170278637771
43.343653250774
49.53560371517028
55.72755417956656
61.91950464396285
68.11145510835914
74.30340557275542
80.4953560371517
86.687306501548
92.87925696594426
99.07120743034056


# 檢視資料

In [25]:
data

,publishAt,isCategoryHeadline,categoryName,title,content,stock,newsId
0,2021/06/30 21:10,1,台股新聞,〈台股盤前要聞〉國巨併購奇力新、星鏈8月覆蓋全球 今日必看財經新聞,關注台股盤前要聞重點，被動元件廠掀整併，國巨宣布併購奇力新，可望貢獻集團營收 15%，獲利估...,"2327,2456,2317,2330,2354,2371,2454,2603,3008,3...",4668195
1,2021/06/30 23:52,1,台股新聞,盤前財經大事搶先讀2021年07月01日,台灣 6 月製造業 PMI日本銀行短觀報告美國上週初領失業金人數股東會：群創、晟德、臻鼎 -...,"2317,2327,2330,2354,2371,2454,2456,3008,3034,3357",4668342
2,2021/06/30 23:53,1,台股盤勢,〈台股盤前〉資金回流電金傳齊揚 Q3大盤續攻靠電子股比重回升,台股在資金回流，電金傳同步點火多頭帶動下，推升指數站上歷史新高收在 17755.46 點，傳...,0,4668292
3,2021/07/01 01:00,1,台股新聞,群益權證小教室：投資人必看的基本面-產業選股,歡迎來到群益權證小教室！「權證」是很適合小資族的投資工具，有機會透過小額資金創造更多收入，今...,"2603,2607,2615,2617,2618,2636",4667787
4,2021/07/01 01:06,1,台股盤勢,【Wen姐筆記】第3季揭序幕 多空搶7大戲先登場？,台股第 2 季上演神鬼奇航，昨日季底作帳幾乎完美謝幕，加權指數以 17755.46 點作收，...,0,4668383
...,...,...,...,...,...,...,...
9654,2021/12/31 04:35,1,台股新聞,金融業走過2021年獲利榮景 升息年存股族先存這4檔,金融業歷經 2021 年業績暢旺的榮景年，也大舉吸引存股族搶進，11 月定期定額個股前十大，...,"2886,2884,5880,2892,2891,2885,5876",4794505
9655,2021/12/31 06:04,1,台股新聞,台灣大併台灣之星、西安封城衝擊記憶體、台股封關 本周大事回顧,中國西安本周傳出疫情爆發，三星、美光等大廠皆在當地設廠，牽動記憶體市況，台灣大周四丟出震撼彈...,"3045,8299,3260,6239,US-MU,2317,2330,2412,2603",4795142
9656,2021/12/31 09:22,1,台股新聞,台光電為擴載板產能 斥21.6億元購入桃園大園工業地產,台灣商用不動產 2021 年年末重大交易頻傳，繼聯德控 - KY(4912-TW) 29 日...,2383,4795215
9657,2021/12/31 11:48,1,台股新聞,迎接2022年 郭台銘分享8字箴言,2021 年即將結束，對於迎接 2022 年，鴻海 (2317-TW) 創辦人郭台銘今 (3...,2317,4795255


# 連結Google Drive並匯出資料

In [26]:
from google.colab import drive
drive.mount('/content/drive')


#輸出檔案

path = '/content/drive/MyDrive/DATA set/'+file_name+'_fast.csv'
data.to_csv(path, index=None)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
